In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 기본 directory 설정
import os
os.chdir('/content/drive/MyDrive/Monthly_Workout')

In [ ]:
# 모듈 불러오기
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
from math import pi

In [ ]:
# 데이터 불러오기
path = './' # 기본 directory 경로에 추가 할 경로

train = pd.read_csv(path + 'train_features.csv')
train_labels = pd.read_csv(path + 'train_labels.csv')
test = pd.read_csv(path + 'test_features.csv')
submission = pd.read_csv(path + 'sample_submission.csv')

In [ ]:
act_list=train.iloc[:,2:].columns
acc_list=['acc_x','acc_y','acc_z']
gy_list=['gy_x','gy_y','gy_z']
act_list

Index(['acc_x', 'acc_y', 'acc_z', 'gy_x', 'gy_y', 'gy_z'], dtype='object')

In [ ]:
# acc 데이터와 gy 데이터로 분할
def sensor_split(data):
    X_acc = []
    X_gy = []

    for i in tqdm(data['id'].unique()):
        temp_acc = np.array(data[data['id'] == i].loc[:,acc_list])
        temp_gy = np.array(data[data['id'] == i].loc[:,gy_list])
        X_acc.append(temp_acc)
        X_gy.append(temp_gy)
      
    X_acc = np.array(X_acc).reshape(-1,600,3)
    X_gy = np.array(X_gy).reshape(-1,600,3)

    return X_acc, X_gy

In [ ]:
# 데이터 증강

def permutation(X, nPerm=4, minSegLength=10):
    X_new = np.zeros(X.shape)
    idx = np.random.permutation(nPerm)
    bWhile = True
    while bWhile == True:
        segs = np.zeros(nPerm+1, dtype=int)
        segs[1:-1] = np.sort(np.random.randint(minSegLength, X.shape[0]-minSegLength, nPerm-1))
        segs[-1] = X.shape[0]
        if np.min(segs[1:]-segs[0:-1]) > minSegLength:
            bWhile = False
    pp = 0
    for ii in range(nPerm):
        x_temp = X[segs[idx[ii]]:segs[idx[ii]+1],:]
        X_new[pp:pp+len(x_temp),:] = x_temp
        pp += len(x_temp)
    return (X_new)

def aug(data, uid, shift):
    shift_data = np.roll(data[uid], shift, axis=0)
    return shift_data
def rolling(data):
    aug_data=[]
    for i in range(data.shape[0]):
        temp=list((aug(data,i,int(random.random()*600))))
        aug_data.append(temp)
    return np.array(aug_data)

In [ ]:
# 데이터 증강 (반복하고 싶은 만큼 조정)
def start_augmentation(train, train_labels):
    # acc, gy 데이터 분할
    X_train_mod=pd.merge(train,train_labels,how='left',on='id')
    X_train_acc, X_train_gy= sensor_split(X_train_mod)

    # 증강시키고 추가할 임시 데이터 복사본
    X_train_acc_temp = X_train_acc.copy()
    X_train_gy_temp = X_train_gy.copy()

    # label 데이터 변환
    y_train = train_labels['label']
    y_train_total = np.append(y_train, y_train, axis=0)

    rep = 4 # 5이상의 경우 reshape 과정에서 reset될 가능성 높음
    for i in range(rep):
        X_train_acc_roll = rolling(X_train_acc_temp)
        X_train_acc_rp = permutation(rolling(X_train_acc_temp)) # rolling + permutation

        X_train_gy_roll = rolling(X_train_gy_temp)
        X_train_gy_rp = permutation(rolling(X_train_gy_temp)) # rolling + permutation

        # 증강시킨 데이터 원래 데이터에 추가
        X_train_acc = np.append(X_train_acc, X_train_acc_roll, axis=0)
        X_train_acc = np.append(X_train_acc, X_train_acc_rp, axis=0)

        X_train_gy = np.append(X_train_gy, X_train_gy_roll, axis=0)
        X_train_gy = np.append(X_train_gy, X_train_gy_rp, axis=0)

        y_train_total = np.append(y_train_total, y_train, axis=0)
        if i != (rep-1): # 마지막 한 번 제외
            y_train_total = np.append(y_train_total, y_train, axis=0)

    return X_train_acc, X_train_gy, y_train_total 

In [ ]:
X_train_acc, X_train_gy, y_train_total = start_augmentation(train, train_labels)

X_train_acc.shape, X_train_gy.shape, y_train_total.shape

100%|██████████| 3125/3125 [00:16<00:00, 195.25it/s]


((40625, 600, 3), (40625, 600, 3), (40625,))

In [ ]:
# np array 형태를 dataframe 으로 변환
def np_to_df(X_train_acc, X_train_gy):
    acc = [e for sl in X_train_acc for e in sl]
    gy = [e for sl in X_train_gy for e in sl]

    df_report_acc = np.stack(acc, axis = 0)
    df_report_gy = np.stack(gy, axis = 0)

    df_acc = pd.DataFrame(df_report_acc, columns= ['acc_x', 'acc_y', 'acc_z']) 
    df_gy = pd.DataFrame(df_report_gy, columns= ['gy_x', 'gy_y', 'gy_z']) 

    # acc, gy 데이터프레임 병합
    df_aug_result = pd.concat([df_acc, df_gy], axis = 1)
    
    return df_aug_result

In [ ]:
train = np_to_df(X_train_acc, X_train_gy)
train

,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112
1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216
2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629
3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013
4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234
...,...,...,...,...,...,...
24374995,-0.797165,-0.407344,-0.111095,-4.135029,-96.765379,-83.078840
24374996,-0.931955,-0.544604,0.065159,-6.710098,-95.525230,-83.847600
24374997,-0.959418,-0.703134,0.274186,-7.480734,-94.221777,-84.194322
24374998,-0.977184,-0.704531,0.219923,-7.768990,-93.280180,-84.567775


In [ ]:
# 가속도
train['acc_t'] = (train['acc_x'] ** 2) + (train['acc_y'] ** 2) + (train['acc_z'] ** 2) ** (1/3)
test['acc_t'] = (test['acc_x'] ** 2) + (test['acc_y'] ** 2) + (test['acc_z'] ** 2) ** (1/3)

train['gy_t'] = (train['gy_x'] ** 2) + (train['gy_y'] ** 2) + (train['gy_z'] ** 2) ** (1/3)
test['gy_t'] = (test['gy_x'] ** 2) + (test['gy_y'] ** 2) + (test['gy_z'] ** 2) ** (1/3)

# Signal 극대화 (peak 캐치 유용)
train['acc_mag'] = (train['acc_x'] ** 2) + (train['acc_y'] ** 2) + (train['acc_z'] ** 2)
test['acc_mag'] = (test['acc_x'] ** 2) + (test['acc_y'] ** 2) + (test['acc_z'] ** 2)

train['gy_mag'] = (train['gy_x'] ** 2) + (train['gy_y'] ** 2) + (train['gy_z'] ** 2)
test['gy_mag'] = (test['gy_x'] ** 2) + (test['gy_y'] ** 2) + (test['gy_z'] ** 2)

In [ ]:
# vector
train['acc_vec'] = np.sqrt((train['acc_x'] ** 2) +(train['acc_y'] ** 2)+(train['acc_z'] ** 2))
test['acc_vec'] = np.sqrt((test['acc_x'] ** 2) +(test['acc_y'] ** 2)+(test['acc_z'] ** 2))

train['gy_vec'] = np.sqrt((train['gy_x'] ** 2) +(train['gy_y'] ** 2)+(train['gy_z'] ** 2))
test['gy_vec'] = np.sqrt((test['gy_x'] ** 2) +(test['gy_y'] ** 2)+(test['gy_z'] ** 2))

# 자이로스코프 무게중심
train['gy_gravity'] = (train['gy_x']+train['gy_y']+train['gy_z'])/3
test['gy_gravity'] = (test['gy_x']+test['gy_y']+test['gy_z'])/3

In [ ]:
# roll & pitch
train['roll'] = np.arctan(train['acc_y']/np.sqrt(train['acc_x'] ** 2 + train['acc_z'] ** 2))
test['roll'] = np.arctan(test['acc_y']/np.sqrt(test['acc_x'] ** 2 + test['acc_z'] ** 2))

train['pitch'] = np.arctan(train['acc_x']/np.sqrt(train['acc_y'] ** 2 + train['acc_z'] ** 2))
test['pitch'] = np.arctan(test['acc_x']/np.sqrt(test['acc_y'] ** 2 + test['acc_z'] ** 2))

train['math_roll'] = np.arctan(- train['acc_x']/np.sqrt(train['acc_y'] ** 2 + train['acc_z'] ** 2)) * (180/pi)
test['math_roll'] = np.arctan(- test['acc_x']/np.sqrt(test['acc_y'] ** 2 + test['acc_z'] ** 2)) * (180/pi)

train['math_pitch'] = np.arctan(train['acc_y']/np.sqrt(train['acc_x'] ** 2 + train['acc_z'] ** 2)) * (180/pi)
test['math_pitch'] = np.arctan(test['acc_y']/np.sqrt(test['acc_x'] ** 2 + test['acc_z'] ** 2)) * (180/pi)

train['gy_roll'] = np.arctan(train['gy_y']/np.sqrt(train['gy_x'] ** 2 + train['gy_z'] ** 2))
test['gy_roll'] = np.arctan(test['gy_y']/np.sqrt(test['gy_x'] ** 2 + test['gy_z'] ** 2))

train['gy_pitch'] = np.arctan(train['gy_x']/np.sqrt(train['gy_y'] ** 2 + train['gy_z'] ** 2))
test['gy_pitch'] = np.arctan(test['gy_x']/np.sqrt(test['gy_y'] ** 2 + test['gy_z'] ** 2))

train['gy_math_roll'] = np.arctan(- train['gy_x']/np.sqrt(train['gy_y'] ** 2 + train['gy_z'] ** 2)) * (180/pi)
test['gy_math_roll'] = np.arctan(- test['gy_x']/np.sqrt(test['gy_y'] ** 2 + test['gy_z'] ** 2)) * (180/pi)

train['gy_math_pitch'] = np.arctan(train['gy_y']/np.sqrt(train['gy_x'] ** 2 + train['gy_z'] ** 2)) * (180/pi)
test['gy_math_pitch'] = np.arctan(test['gy_y']/np.sqrt(test['gy_x'] ** 2 + test['gy_z'] ** 2)) * (180/pi)

print(train.shape)
train

(24375000, 21)


,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,acc_t,gy_t,acc_mag,gy_mag,acc_vec,gy_vec,gy_gravity,roll,pitch,math_roll,math_pitch,gy_roll,gy_pitch,gy_math_roll,gy_math_pitch
0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112,1.767177,943.603223,1.508855,1936.968046,1.228355,44.010999,-20.938910,-0.146550,1.380095,-79.073624,-8.396683,-0.767200,-0.013443,0.770209,-43.957305
1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216,2.019429,1540.494535,1.731038,2153.327355,1.315689,46.403958,-21.254406,-0.151814,1.364146,-78.159835,-8.698302,-1.003661,0.006532,-0.374246,-57.505519
2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629,2.140496,1968.439749,1.804277,2585.867212,1.343234,50.851423,-24.253157,-0.145773,1.330405,-76.226605,-8.352177,-1.050508,-0.071194,4.079137,-60.189701
3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013,2.084285,2889.189647,1.771478,3633.811839,1.330969,60.281107,-26.112956,-0.173958,1.331665,-76.298763,-9.967037,-1.095444,0.045021,-2.579501,-62.764338
4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234,2.094771,3380.757973,1.777076,4153.376248,1.333070,64.446693,-27.193696,-0.141316,1.350615,-77.384519,-8.096783,-1.116383,0.066565,-3.813876,-63.964038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24374995,-0.797165,-0.407344,-0.111095,-4.135029,-96.765379,-83.078840,1.032499,9399.676821,0.813743,16282.730691,0.902077,127.603804,-61.326416,-0.468515,-1.083708,62.091904,-26.843933,-0.860742,-0.032411,1.857007,-49.316910
24374996,-0.931955,-0.544604,0.065159,-6.710098,-95.525230,-83.847600,1.327061,9189.252044,1.169380,16200.515121,1.081379,127.281244,-62.027643,-0.527784,-1.038850,59.521705,-30.239794,-0.848826,-0.052743,3.021958,-48.634155
24374997,-0.959418,-0.703134,0.274186,-7.480734,-94.221777,-84.194322,1.836930,8952.914403,1.490059,16022.388483,1.220680,126.579574,-61.965611,-0.613850,-0.904264,51.810533,-35.170986,-0.839588,-0.059134,3.388101,-48.104839
24374998,-0.977184,-0.704531,0.219923,-7.768990,-93.280180,-84.567775,1.815597,8780.815667,1.499617,15913.257787,1.224589,126.147762,-61.872315,-0.612996,-0.923918,52.936576,-35.122057,-0.832256,-0.061625,3.530877,-47.684732


In [ ]:
# # Scaling 하기 전 Outlier 제거 작업
# def get_outlier(df, column, weight=1.5):
#   # target 값과 상관관계가 높은 열을 우선적으로 진행
#   quantile_25 = np.percentile(df[column].values, 25)
#   quantile_75 = np.percentile(df[column].values, 75)

#   IQR = quantile_75 - quantile_25
#   IQR_weight = IQR*weight
  
#   lowest = quantile_25 - IQR_weight
#   highest = quantile_75 + IQR_weight
  
#   outlier_idx = df[column][ (df[column] < lowest) | (df[column] > highest) ].index
#   return outlier_idx

# # 함수 사용해서 이상치 값 삭제
# name_features = train.columns.values.tolist()
# for i in name_features:
#     outlier_idx = get_outlier(df=train, column=i)
#     train.drop(outlier_idx, axis=0, inplace=True)
# train.shape

In [ ]:
# Scaling 원하는 걸로 사용
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer

scaler = RobustScaler()
train = scaler.fit_transform(train)
test.drop(['id', 'time'], axis=1, inplace=True)
test = scaler.transform(test)
train

array([[ 1.93489817,  0.03657354, -0.31330862, ...,  0.05924213,
        -0.05924213, -0.71658616],
       [ 2.01969699,  0.00673528, -0.37289479, ...,  0.07768724,
        -0.07768724, -0.94204715],
       [ 2.0372705 ,  0.01260973, -0.49722803, ...,  0.0059123 ,
        -0.0059123 , -0.98671563],
       ...,
       [-0.31522429, -0.76068882,  0.42743318, ...,  0.01704969,
        -0.01704969, -0.78560684],
       [-0.33368375, -0.76281458,  0.33232805, ...,  0.01474858,
        -0.01474858, -0.77861568],
       [-0.35269944, -0.51028997,  0.18474317, ...,  0.00900187,
        -0.00900187, -0.77374585]])

In [ ]:
import tensorflow as tf 
from keras.models import Sequential
from keras.layers import Dropout, LSTM, Input
from keras.layers import TimeDistributed
from keras.layers import Activation, GlobalAveragePooling1D
from keras.layers import Dense, Flatten, BatchNormalization
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from keras.models import load_model
from keras.layers.merge import concatenate

In [ ]:
len_features = train.shape[1] # feature 갯수
X = train.reshape(-1, 600, len_features)
X.shape

(40625, 600, 21)

In [ ]:
y = to_categorical(y_train_total) 
y.shape

(40625, 61)

In [ ]:
epochs, batch_size = 50, 64 # Ram 24GB 기준 256 이상 reset될 가능성 높음

In [ ]:
test_X = test.reshape(-1, 600, len_features)
test_X.shape

(782, 600, 21)

In [ ]:
class Models:
    # 각종 size 다양하게 custom해서 stacking 쌓아 올리기
    def define_model_0():
        model = Sequential()
        model.add(Conv1D(filters=32, kernel_size=9, padding='same', input_shape=(600, len_features)))
        model.add(BatchNormalization())
        model.add(Activation(activation='relu'))
        model.add(Dropout(0.3))

        model.add(Conv1D(filters=32, kernel_size=6, padding='same'))
        model.add(BatchNormalization())
        model.add(Activation(activation='relu'))
        model.add(Dropout(0.4))

        model.add(Conv1D(filters=32, kernel_size=3, padding='same'))
        model.add(BatchNormalization())
        model.add(Activation(activation='relu'))
        model.add(Dropout(0.5))

        model.add(GlobalAveragePooling1D())
        model.add(Dense(61, activation='softmax'))

        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

        return model

    def define_model_1():
        model = Sequential()
        model.add(Conv1D(filters=64, kernel_size=3, padding='same', input_shape=(600, len_features)))
        model.add(BatchNormalization())
        model.add(Activation(activation='relu'))
        model.add(Dropout(0.3))

        model.add(Conv1D(filters=64, kernel_size=3, padding='same'))
        model.add(BatchNormalization())
        model.add(Activation(activation='relu'))
        model.add(Dropout(0.3))

        model.add(Conv1D(filters=64, kernel_size=3, padding='same'))
        model.add(BatchNormalization())
        model.add(Activation(activation='relu'))
        model.add(Dropout(0.3))

        model.add(GlobalAveragePooling1D())
        model.add(Dense(61, activation='softmax'))

        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

        return model

    def define_model_2():
        model = Sequential()
        model.add(Conv1D(filters=32, kernel_size=3, padding='same', input_shape=(600, len_features)))
        model.add(BatchNormalization())
        model.add(Activation(activation='relu'))
        model.add(Dropout(0.5))

        model.add(Conv1D(filters=64, kernel_size=3, padding='same'))
        model.add(BatchNormalization())
        model.add(Activation(activation='relu'))
        model.add(Dropout(0.5))

        model.add(Conv1D(filters=64, kernel_size=3, padding='same'))
        model.add(BatchNormalization())
        model.add(Activation(activation='relu'))
        model.add(Dropout(0.5))

        model.add(GlobalAveragePooling1D())
        model.add(Dense(61, activation='softmax'))

        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

        return model

    def define_model_3():
        model = Sequential()
        model.add(Conv1D(filters=64, kernel_size=6, padding='same', input_shape=(600, len_features)))
        model.add(BatchNormalization())
        model.add(Activation(activation='relu'))
        model.add(Dropout(0.3))

        model.add(Conv1D(filters=64, kernel_size=3, padding='same'))
        model.add(BatchNormalization())
        model.add(Activation(activation='relu'))
        model.add(Dropout(0.3))

        model.add(Conv1D(filters=128, kernel_size=3, padding='same'))
        model.add(BatchNormalization())
        model.add(Activation(activation='relu'))
        model.add(Dropout(0.3))

        model.add(GlobalAveragePooling1D())
        model.add(Dense(61, activation='softmax'))

        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

        return model

    def define_model_4():
        input_layer = Input(600, len_features)

        model = Sequential()
        model.add(Conv1D(filters=64, kernel_size=6, padding='same', input_shape=(600, len_features)))
        model.add(BatchNormalization())
        model.add(Activation(activation='relu'))
        model.add(Dropout(0.5))

        model.add(Conv1D(filters=128, kernel_size=3, padding='same'))
        model.add(BatchNormalization())
        model.add(Activation(activation='relu'))
        model.add(Dropout(0.5))

        model.add(Conv1D(filters=128, kernel_size=3, padding='same'))
        model.add(BatchNormalization())
        model.add(Activation(activation='relu'))
        model.add(Dropout(0.3))

        model.add(GlobalAveragePooling1D())
        model.add(Dense(61, activation='softmax'))

        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

        return model

In [23]:
# 위의 모델들 학습하면서 stacking
for i in range(5):
    model = getattr(Models, f'define_model_{i}')()
    checkpoint_path = "checkpoint/cp.ckpt"
    cp_callback = ModelCheckpoint(filepath=checkpoint_path, monitor='loss', 
                                verbose=1, save_weights_only=True, 
                                save_best_only=True, mode='min')
    early_stopping = EarlyStopping(monitor='loss', patience=20, mode='min')
    model.fit(X, y, epochs=epochs, batch_size=batch_size, 
            validation_split=0.2, callbacks=[early_stopping, cp_callback])
    model.save(f'models/model_{i}.h5')
    tf.keras.backend.clear_session()

Epoch 1/100
508/508 [==============================] - 21s 10ms/step - loss: 2.6695 - accuracy: 0.4770 - val_loss: 2.5378 - val_accuracy: 0.4932

Epoch 00001: loss improved from inf to 2.66953, saving model to checkpoint/cp.ckpt
Epoch 2/100
508/508 [==============================] - 4s 9ms/step - loss: 2.3027 - accuracy: 0.5158 - val_loss: 2.4664 - val_accuracy: 0.5068

Epoch 00002: loss improved from 2.66953 to 2.30271, saving model to checkpoint/cp.ckpt
Epoch 3/100
508/508 [==============================] - 4s 9ms/step - loss: 2.1895 - accuracy: 0.5354 - val_loss: 2.4014 - val_accuracy: 0.5199

Epoch 00003: loss improved from 2.30271 to 2.18951, saving model to checkpoint/cp.ckpt
Epoch 4/100
508/508 [==============================] - 4s 9ms/step - loss: 2.1030 - accuracy: 0.5513 - val_loss: 2.3607 - val_accuracy: 0.5265

Epoch 00004: loss improved from 2.18951 to 2.10300, saving model to checkpoint/cp.ckpt
Epoch 5/100
508/508 [==============================] - 4s 9ms/step - loss: 2.0

In [24]:
# 저장한 모델 불러오기
for i in range(5):
    globals()[f'model{i}'] = load_model(f'models/model_{i}.h5')

In [25]:
model0._name = 'Client1'
model1._name = 'Client2'
model2._name = 'Client3'
model3._name = 'Client4'

In [26]:
inputs = Input(shape=(600, len_features))

merge = concatenate([model0(inputs), model1(inputs), model2(inputs), model3(inputs), model4(inputs)])
hidden = Dense(10, activation='relu')(merge)
output = Dense(61, activation='softmax')(hidden)

model = Model(inputs=inputs, outputs=output)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [27]:
checkpoint_path = "checkpoint/cp.ckpt"
cp_callback = ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss', verbose=1, save_weights_only=True, save_best_only=True, mode='min')

early_stopping = EarlyStopping(monitor='val_loss', patience=20, mode='min')
model.fit(X, y, epochs=epochs, batch_size=batch_size, validation_split=0.02, callbacks=[early_stopping, cp_callback])

Epoch 1/100
623/623 [==============================] - 31s 43ms/step - loss: 2.6270 - accuracy: 0.4936 - val_loss: 2.6620 - val_accuracy: 0.4859

Epoch 00001: val_loss improved from inf to 2.66198, saving model to checkpoint/cp.ckpt
Epoch 2/100
623/623 [==============================] - 26s 42ms/step - loss: 2.0311 - accuracy: 0.5765 - val_loss: 2.6934 - val_accuracy: 0.4723

Epoch 00002: val_loss did not improve from 2.66198
Epoch 3/100
623/623 [==============================] - 26s 42ms/step - loss: 1.8218 - accuracy: 0.6354 - val_loss: 2.7481 - val_accuracy: 0.4551

Epoch 00003: val_loss did not improve from 2.66198
Epoch 4/100
623/623 [==============================] - 26s 42ms/step - loss: 1.7073 - accuracy: 0.6624 - val_loss: 2.7611 - val_accuracy: 0.4588

Epoch 00004: val_loss did not improve from 2.66198
Epoch 5/100
623/623 [==============================] - 26s 42ms/step - loss: 1.6343 - accuracy: 0.6844 - val_loss: 2.8006 - val_accuracy: 0.4453

Epoch 00005: val_loss did not 

In [ ]:
prediction = model.predict(test_X)
prediction.shape

In [ ]:
submission

In [ ]:
submission.iloc[:,1:]=prediction

In [ ]:
submission

In [32]:
submission.to_csv('submission/just_cnn_stacked_5(jun).csv', index=False)